<h1>Transfer Learning and Fine-Tuning in PyTorch</h1>

<b>Transfer learning</b> leverages pre-trained models to improve the performance of a model on a new, but related task.<br>
This technique is particularly useful when you have a limited amount of data for your specific task. <br>
PyTorch provides several pre-trained models in the <b>torchvision.models</b> module, which can be fine-tuned for various tasks.

<h4>What is Transfer Learning?</h4>
Transfer learning involves using a model trained on a large dataset for a new, but similar task.<br>
The idea is to utilize the feature representations learned by the model on the original task and adapt them to the new task. <br>
This typically involves:
<ul>
<li><b>Feature Extraction:</b> Using the pre-trained model as a fixed feature extractor.</li>
<li><b>Fine-Tuning:</b> Updating the weights of the pre-trained model to fit the new task.</li>
</ul>

<h4>Steps for Transfer Learning</h4>
<ol>
<li><b>Load a Pre-trained Model:</b> Use models available in <b>torchvision.models</b>.</li>
<li><b>Modify the Model:</b> Adapt the model to your specific task.</li>
<li><b>Prepare the Data:</b> Use appropriate data loaders for your task.</li>
<li><b>Train the Model:</b> Fine-tune the model on your dataset.</li>
<li><b>Evaluate the Model:</b> Assess the performance on a test set.</li>
</ol>

<h4>Transfer Learning with a Pre-Trained ResNet Model on CIFAR-10</h4>
 fine-tuning a pre-trained ResNet model on the CIFAR-10 dataset

<h5> Step 1 : Import libraries </h5>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader

<h5> Step 2: Operations on Image </h5>

In [148]:
# Define transformations for training and testing
transform = transforms.Compose([
    transforms.Resize(256),                  # Resize images to 256x256
    transforms.CenterCrop(224),              # Crop center 224x224
    transforms.ToTensor(),                   # Convert image to tensor
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalize using CIFAR-10 stats
])

<h5> Step 3: Load the dataset</h5>

In [151]:
# Load CIFAR-10 dataset
full_trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


<h5> Step 4: Create a subset of datast </h5>
As we are running on CPU, the smaller set of dataset is preferable. Due to that, we are making subset. 

In [158]:
# Create a subset of 100 images from the training dataset
indices = list(range(100))  # Adjusted to include 100 samples
print("indices = ",indices)

trainset = Subset(full_trainset, indices)
batch_size = 10
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)

indices =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [160]:
# Load CIFAR-10 test dataset
full_testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
indices = list(range(10))
print("indices = ",indices)

testset = Subset(full_testset, indices)
testloader = DataLoader(testset, batch_size=1, shuffle=False)

Files already downloaded and verified
indices =  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


<h5>Step 5: Use pre=trained ResNet model</h5>
Transfer learning leverages a model that has been pre-trained on a large dataset (e.g., ImageNet) to solve a related task. Instead of training a model from scratch, you use the learned features of a pre-trained model to improve performance on a new dataset with potentially fewer data.

In [169]:
# Load the pre-trained ResNet model
model = models.resnet18(pretrained=True)

<h5> Step 6: Modify the pre-trained ResNet model</h5>

<b>Fine-tuning</b> involves adjusting the pre-trained model on a new dataset by continuing the training process. The goal is to adapt the model’s features to the specific characteristics of the new dataset.<br>

The <b>final layer of the pre-trained ResNet model</b> is modified to match the number of classes in the new dataset (CIFAR-10). This step adapts the model to the specific task.

In [181]:
# Modify the model for CIFAR-10 (10 classes)
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

In [183]:
# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

<h5>Step 7: Loss function and Optimizer </h5>

In [185]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [192]:
# Training loop
num_epochs = 10

<h5> Step 8: Training </h5>

In [195]:
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()

        running_loss += loss.item()

    # Print average loss for the epoch
    avg_loss = running_loss / len(trainloader)
    print(f'End of Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}')

print('Finished Training')

Epoch 1/10
End of Epoch 1, Average Loss: 0.2321
Epoch 2/10
End of Epoch 2, Average Loss: 0.1961
Epoch 3/10
End of Epoch 3, Average Loss: 0.1499
Epoch 4/10
End of Epoch 4, Average Loss: 0.1348
Epoch 5/10
End of Epoch 5, Average Loss: 0.0735
Epoch 6/10
End of Epoch 6, Average Loss: 0.1115
Epoch 7/10
End of Epoch 7, Average Loss: 0.0705
Epoch 8/10
End of Epoch 8, Average Loss: 0.0534
Epoch 9/10
End of Epoch 9, Average Loss: 0.0485
Epoch 10/10
End of Epoch 10, Average Loss: 0.0525
Finished Training


<h5> Step 9: Testing </h5>

In [ ]:
# Evaluate the model
correct = 0
total = 0

with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10 test images: {100 * correct / total:.2f}%')